# NDVI Data Merger

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/merge_ndvi_data.ipynb)

This notebook merges NDVI data from multiple years (2013-2023) into separate files for each area.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import glob
import os

# Define the base directory where CSV files are stored
base_dir = '/content/drive/MyDrive/earthengine/csv'

# Define the years and areas to process
years = [2013, 2015, 2017, 2019, 2021, 2023]
areas = range(5, 9)  # Areas 5 through 8

# Process each area separately
for area in areas:
    print(f"Processing Area {area}...")
    
    # Start with the first year as base
    base_file = f"{base_dir}/NDVI_Annual_{years[0]}_Area_{area}_vectorized.csv"
    if not os.path.exists(base_file):
        print(f"Warning: Base file not found for Area {area}")
        continue
        
    # Read the base year data
    base_df = pd.read_csv(base_file)
    
    # Rename the year column to include the year
    base_df = base_df.rename(columns={'y2013': f'y{years[0]}'})
    
    # Merge with subsequent years
    for year in years[1:]:
        year_file = f"{base_dir}/NDVI_Annual_{year}_Area_{area}_vectorized.csv"
        
        if not os.path.exists(year_file):
            print(f"Warning: File not found for year {year}, Area {area}")
            continue
            
        # Read the year's data
        year_df = pd.read_csv(year_file)
        
        # Rename the year column
        year_df = year_df.rename(columns={f'y{year}': f'y{year}'})
        
        # Merge with base dataframe
        base_df = pd.merge(
            base_df,
            year_df[['pixel_id', f'y{year}']],
            on='pixel_id',
            how='outer'
        )
    
    # Save the merged data
    output_file = f"{base_dir}/NDVI_Annual_AllYears_Area_{area}.csv"
    base_df.to_csv(output_file, index=False)
    print(f"Saved merged data for Area {area} to {output_file}")

print("\nProcessing complete!")